In [15]:
import pandas as pd
import numpy as np
df=pd.read_csv('../Emission_computing/final_ingredients_emissions.csv')
import re
import nltk
from nltk.stem import WordNetLemmatizer 


In [3]:
def fill_empties(output_dict):
    output_df=pd.DataFrame(output_dict)
    output_df.metric=output_df.metric.astype(str)
    for item in missing_values:
        output_df.metric.replace(item, "g", inplace=True)
    return output_df

In [9]:
def match_ingredients(output_df):
    final_dict={'ingredient':[], 'emission':[]}
    missing_ingredients={'missing_ingredients':[]}
    for ingredient in output_df['ingredients']:
        
        # 1. Lemmatize 
        ingredient_words=re.split('\s+', ingredient)
        lemmatizer = WordNetLemmatizer()
        lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in ingredient_words])
        
        # 2. Try the whole lemmatized sentence
        if df[df['ingredient'].str.match(r'.*'+str(lemmatized_output)+'.*')== True].ingredient.values.size>0:
                final_dict['ingredient'].append(df[df['ingredient'].str.match(r'.*'+str(lemmatized_output)+'.*')== True].ingredient.values[0])
                final_dict['emission'].append(df[df['ingredient'].str.match(r'.*'+str(lemmatized_output)+'.*')== True].emissions.values[0])
        
        # 3. If the lemmatized output does not work, try the words
        else:
            for word in ingredient_words:
                try:
                    final_dict['ingredient'].append(df[df['ingredient'].str.match(r'.*'+str(word)+'.*')== True].ingredient.values[0])
                    final_dict['emission'].append(df[df['ingredient'].str.match(r'.*'+str(word)+'.*')== True].emissions.values[0])
                except IndexError:
                    missing_ingredients['missing_ingredients'].append(ingredient)
            
    final_df=pd.DataFrame(final_dict)
    
    
    # Adding the columns "value" and metric from output_dict
#     final_df=final_df.merge(output_df,on="ingredient")
#     final_df['value']=output_df['value']
#     final_df['metric']=output_df['metric']
    return final_df,missing_ingredients

In [10]:
ingredients_df=pd.read_csv('../Emission_computing/clean_ingredients.csv')

In [11]:
ingredients_df

,ingredients
0,donuts
1,milk chocolate
2,milk
3,espresso
4,vanilla
...,...
241,capers
242,anchovy fillets
243,tabasco sauce
244,worcestershire sauce


In [17]:
final_df,missing_ingredients=match_ingredients(ingredients_df)

In [18]:
final_df

,ingredient,emission
0,buttermilk,1100
1,chocolate,5763
2,buttermilk,1100
3,espresso,706
4,oatly oat drink - vanilla,430
...,...,...
255,dried peas (shelled),2388
256,thyme,1133
257,lime,1900
258,apple sauce,2146


In [20]:
pd.DataFrame(missing_ingredients)

,missing_ingredients
0,donuts
1,chili
2,fresh ginger
3,cilantro
4,chile paste
...,...
130,capers
131,anchovy fillets
132,anchovy fillets
133,tabasco sauce
